# 第6章
モデルのハイパーパラメータのチューニングや特徴量の選択によってモデルの精度を高めるテクニックを学ぶ。

---
ソースコードは以下から引用しています: https://github.com/ghmagazine/kagglebook/tree/master/ch06

ライセンス: https://github.com/ghmagazine/kagglebook/blob/master/LICENSE

## hyperopt（ベイズ最適化）でGBDTのパラメータチューニングを行う

### データの準備

In [1]:
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv('data/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('data/sample-data/test_preprocessed.csv')

# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

### モデルの準備

In [2]:
# xgboostによる学習・予測を行うクラス
import xgboost as xgb


class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

### チューニングしたいパラメータを引数にとり、最小化したい評価指標のスコアを返す関数を作成
モデルを引数のパラメータで学習させ、バリデーションデータへの予測を行い、評価指標のスコアを計算する処理を行う

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import log_loss


def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}

### 探索するパラメータ空間を定義 

In [4]:
# 探索するパラメータの空間を指定する
space = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1),
}

### パラメータ探索の実行

In [5]:
# hyperoptによるパラメータ探索の実行
# 探索回数（100回程度で十分）
max_evals = 10  #　ここでは10で試す
trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

[0]	train-error:0.1408	eval-error:0.1572
[1]	train-error:0.133733	eval-error:0.1532
[2]	train-error:0.125867	eval-error:0.1504
[3]	train-error:0.1212	eval-error:0.1444
[4]	train-error:0.1172	eval-error:0.1424
[5]	train-error:0.110933	eval-error:0.1392
[6]	train-error:0.107467	eval-error:0.1384
[7]	train-error:0.104933	eval-error:0.1356
[8]	train-error:0.099333	eval-error:0.1256
[9]	train-error:0.0956	eval-error:0.1268
params: {'gamma': 0.30000000000000004, 'max_depth': 5, 'min_child_weight': 1.0}, logloss: 0.3077
[0]	train-error:0.1116	eval-error:0.1428
[1]	train-error:0.097733	eval-error:0.1452
[2]	train-error:0.091333	eval-error:0.1396
[3]	train-error:0.081867	eval-error:0.1316
[4]	train-error:0.076533	eval-error:0.1268
[5]	train-error:0.068533	eval-error:0.1212
[6]	train-error:0.061733	eval-error:0.1184
[7]	train-error:0.054667	eval-error:0.1192
[8]	train-error:0.0508	eval-error:0.1164
[9]	train-error:0.045067	eval-error:0.1148
params: {'gamma': 0.1, 'max_depth': 9, 'min_child_weigh

{'gamma': 0.2, 'max_depth': 8.0, 'min_child_weight': 1.0}

### 結果の出力

In [6]:
# 記録した情報からパラメータとスコアを出力する
# （trialsからも情報が取得できるが、パラメータの取得がやや行いづらいため）
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')

best params:{'gamma': 0.2, 'max_depth': 8, 'min_child_weight': 1.0}, score:0.2806


## GBDTで特徴量の重要度を求める

In [7]:
# train_xは学習データ、train_yは目的変数
train = pd.read_csv('data/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']

import xgboost as xgb

# xgboost
dtrain = xgb.DMatrix(train_x, label=train_y)
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
num_round = 50
model = xgb.train(params, dtrain, num_round)

# 重要度の上位を出力する
fscore = model.get_score(importance_type='total_gain')  # デフォルトの頻度ではなくゲインを見る
fscore = sorted([(k, v) for k, v in fscore.items()], key=lambda tpl: tpl[1], reverse=True)
print('xgboost importance')
print(fscore[:5])

xgboost importance
[('weight', 2614.0292872053), ('medical_info_a1', 2240.9029885495024), ('height', 1973.3420535613589), ('age', 1442.8326779044812), ('medical_info_a2', 1150.6861457969187)]
